In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Komoran

1. 데이터 로드
2. 데이터 튜닝
3. 데이터 분할
4. 토큰화
5. 벡터화
6. 모델 학습
7. 평가

#### 1. 데이터 로드

In [2]:
# 데이터 로드
df = pd.read_csv("../data/ratings_train.txt", sep='\t')
df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


#### 2. 데이터 튜닝

In [3]:
# 필요 없는 컬럼 제외
# id 컬럼 - 모두 독립적이고 중복이 없는 유일한 값이므로
df.drop('id', axis=1, inplace=True)

In [4]:
# 결측치 존재 여부 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   document  149995 non-null  object
 1   label     150000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.3+ MB


In [5]:
# 전체 15만 개의 데이터 중 결측치 5개
# 굉장히 적은 양이므로 제거
df.dropna(inplace=True)

In [6]:
# 리뷰 데이터 중 중복된 문장 존재 여부 확인
# value_counts()
df['document'].value_counts()

document
굿                                                181
good                                              92
최고                                                85
쓰레기                                               79
별로                                                66
                                                ... 
굿바이 레닌 표절인것은 이해하는데 왜 뒤로 갈수록 재미없어지냐                 1
이건 정말 깨알 캐스팅과 질퍽하지않은 산뜻한 내용구성이 잘 버무러진 깨알일드!!♥      1
약탈자를 위한 변명, 이라. 저놈들은 착한놈들 절대 아닌걸요.                 1
나름 심오한 뜻도 있는 듯. 그냥 학생이 선생과 놀아나는 영화는 절대 아님          1
흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나                  1
Name: count, Length: 146182, dtype: int64

In [7]:
# 중복으로 만들어져 있는 리뷰 문자들은 제거해 과적합 방지
df.drop_duplicates('document', inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146182 entries, 0 to 149999
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   document  146182 non-null  object
 1   label     146182 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.3+ MB


In [9]:
# 학습 데이터와 검증 데이터로 분할
X = df['document'].values
Y = df['label'].values

# 분류 데이터이므로 label의 비율을 유지 (stratify)
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, stratify=Y, random_state=42
)

In [10]:
# X의 데이터가 문자이므로 토큰화
komoran = Komoran()

# 조건 설정
# 사용할 품사 선택
allow_pos = ['NNP', 'NNG', 'VV', 'VA', 'SL', 'MAG']
# 사용하지 않을 단어 선택
stop_word = ['하다', '되다']
# 최소 글자 수 제한
len_word = 2

# 토큰화 함수 정의
def tokenize(text):
    # 결과를 리스트에 되돌려주기 위해 빈 리스트 생성
    tokens = []
    for word, pos in komoran.pos(text):
        # 조건 1. 사용할 품사에 포함되어 있다면
        # 조건 2. 금지어에 포함되어 있지 않다면
        # 조건 3. 문자의 길이가 len_word보다 크거나 같다면
        if pos in allow_pos and word not in stop_word and len(word) >= len_word:
            # 3개의 조건을 모두 만족하는 단어를 tokens에 추가
            tokens.append(word)
    return tokens

In [11]:
# 벡터화 객체 생성
# 단어의 중요도를 판단하는 벡터화 class 로드
# 벡터화 - 자연어 데이터에서의 스케일링과 비슷한 작업
vectorizer = TfidfVectorizer(
    tokenizer= tokenize,
    ngram_range= (1, 1),
    min_df= 3,      # 3회 이상 나온 단어들을 기준으로 중요도 판단
    lowercase= False
)

In [12]:
# train 데이터를 이용하여 fit_transform()
# test 데이터는 transform() --> 데이터 누수 방지
X_train_vec = vectorizer.fit_transform(X_train)
print(len(vectorizer.get_feature_names_out()))

c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


13575


In [13]:
X_test_vec = vectorizer.transform(X_test)
print(len(vectorizer.get_feature_names_out()))

13575


- train, test 데이터 모두에 fit을 하면 두 개의 데이터에서 단어를 추출했을 때 다른 값을 보인다. (**데이터의 누수**)
- train 데이터를 이용해 fit 하고, test 데이터에 transform 작업

In [14]:
# 출력 결과가 너무 커서 메모리 용량 부족으로 MemoryError
# X_train_vec.toarray()

In [15]:
# X_train_vec, Y_train 데이터를 이용하여
model = LinearSVC(C= 1.0)

In [16]:
# 모델에 학습
# 13,575개의 컬럼에서 label 0, 1 사이의 규칙을 찾아내는 과정
model.fit(X_train_vec, Y_train)

,penalty,'l2'
,loss,'squared_hinge'
,dual,'auto'
,tol,0.0001
,C,1.0
,multi_class,'ovr'
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,verbose,0
,random_state,None


In [17]:
pred = model.predict(X_test_vec)
pred

array([1, 0, 0, ..., 0, 1, 1], shape=(29237,))

In [18]:
acc = accuracy_score(pred, Y_test)
f1 = f1_score(pred, Y_test)
print(f'정확도: {acc :.4f}, F1 score: {f1 :.4f}')

정확도: 0.7735, F1 score: 0.7669


In [19]:
# 모델의 성능을 올리기 위해 최적의 파라미터를 찾는 과정
# Pipeline, GridsearchCV, StratifiedKFold
# 파이프라인 생성
pipe = Pipeline(
    [
        # 첫 번째 step - 벡터화
        (
            'vectorizer', TfidfVectorizer(
                # 고정으로 사용할 매개변수의 값들을 지정
                lowercase= False,
                max_df= 0.95,       # 너무 자주 등장하는 단어는 배제
                sublinear_tf= True  # tf를 log( 1 + tf )로 스케일
            )
        ),
        # 두 번째 step - 모델에 학습
        (
            'clf', LinearSVC()
        )
    ]
)

In [20]:
# pipe에서 사용할 매개변수의 값들 지정
param_grid = {
    'vectorizer__ngram_range': [ (1, 1), (1, 2) ],
    'vectorizer__min_df': [ 3, 5 ],
    'clf__C': [ 0.9, 1.0 ]
}

In [21]:
# 교차 검증 폴드화
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

In [22]:
grid = GridSearchCV(
    estimator= pipe,
    param_grid= param_grid,
    scoring= 'f1_macro',
    cv = cv,
    n_jobs= -1,
    verbose= 1
)

In [23]:
# GridSearch를 이용한 학습
grid.fit(X_train, Y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


,estimator,Pipeline(step...LinearSVC())])
,param_grid,"{'clf__C': [0.9, 1.0], 'vectorizer__min_df': [3, 5], 'vectorizer__ngram_range': [(1, ...), (1, ...)]}"
,scoring,'f1_macro'
,n_jobs,-1
,refit,True
,cv,StratifiedKFo... shuffle=True)
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,input,'content'


In [24]:
print("최적의 매개변수: ", grid.best_params_)
print('최적의 f1 score: ', round(grid.best_score_, 4))

최적의 매개변수:  {'clf__C': 0.9, 'vectorizer__min_df': 3, 'vectorizer__ngram_range': (1, 2)}
최적의 f1 score:  0.7934


---
#### 실습문제
- 토큰화
    - komoran 사용
    - 품사는 ('NNP', 'NNG', 'VV', 'VA', 'MAG', 'SL')만 사용
    - 단어의 길이는 2 이상
    - 금지어는 ('하다', '되다')
- 벡터화
    - TfidVectorizer 사용
    - 고정 매개변수는
        - tokenizer = komoran
        - lowercase = False
        - max_df = 0.95
        - sublinear_tf = True
- 분류 모델
    - LogisticRegression 사용
    - 고정 매개변수는
        - n_jobs = -1
        - class_weight = 'balanced'
- Pipeline 생성
    - 벡터화 ('vector')
    - 분류 모델 ('clf')
- 교차 검증
    - StartifiedKFold
        - 폴드의 개수는 4
        - shuffle = True
        - random_state = 42
- Gridsearch
    - 벡터화
        - ngram_range는 (1, 1), (2, 2)
        - min_df는 3, 5
    - 분류 모델
        - max_iter는 800, 1000
        - C는 1.0, 2.0

In [25]:
# 1. 토큰화
# 2. 벡터화
# 3. 분류 모델
# 4. 파이프라인 생성
# 5. 교차 검증
# 6-1. 그리드 서치 - 벡터화
# 6-2. 그리드 서치 - 분류 모델

In [26]:
from sklearn.linear_model import LogisticRegression

In [31]:
# 1. 토큰화
komoran = Komoran()

# 조건 설정
# 사용할 품사 선택
allow_pos = ['NNP', 'NNG', 'VV', 'VA', 'MAG', 'SL']
# 사용하지 않을 단어 선택
stop_word = ['하다', '되다']
# 최소 글자 수 제한
len_word = 2

# 토큰화 함수 정의
def tokenize(text):
    # 필터링된 단어들을 저장할 공간
    tokens = []
    for word, pos in komoran.pos(text):
        # word: komoran을 이용해서 만들어진 단어
        # pos: 품사 (Part Of Speech)
        if pos in allow_pos and word not in stop_word and len(word) >= len_word:
            # 위의 조건을 모두 만족하는 단어를 tokens에 추가
            tokens.append(word)
    return tokens

# 2. 벡터화
vectorizer_komoran = TfidfVectorizer(
    tokenizer= tokenize,        # komoran을 사용해 나온 단어들을 tokenize에서 필터링
    lowercase= False,
    max_df= 0.95,
    sublinear_tf= True
)

# 3. 분류 모델
model_logi = LogisticRegression(
    n_jobs = -1,
    class_weight = 'balanced'
)

# 4. 파이프라인 생성
pipe_logi = Pipeline(
    [
        # 첫 번째 step - 벡터화
        (
            # 'vector', 'clf'와 같이 이름을 지정해두면 GridSearchCV에서 편하게 사용 가능
            'vector', vectorizer_komoran
        ),
        # 두 번째 step - 모델에 학습
        (
            'clf', model_logi
        )
    ]
)

# 5. 교차 검증(계층화 교차 검증) 4회
cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)

# GridSearch에서 사용할 파라미터 값들
# pipe에서 생성한 key값들과 매개변수 명으로 지정
# {pipe_key}__{매개변수명}
    # 매개변수명에서 언더바(_) 1개인 경우가 흔하므로 언더바 2개로 나눠 key값과 매개변수명 구분하는 것
param_grid = {
    # ↓ pipe에서 'vector'의 key를 가진 객체를 찾아 매개변수 ngram_range 값을 변경한다는 뜻
    'vector__ngram_range': [ (1,1), (1,2) ],
    'vector__min_df': [3, 5],
    'clf__max_iter': [800, 1000],
    'clf__C': [1.0, 2.0]
}

# 6-1. 그리드 서치 - 벡터화
grid_logi = GridSearchCV(
    estimator= pipe_logi,
    param_grid= param_grid,
    cv= cv,
    scoring= 'f1_macro',
    verbose= 1,
    # n_jobs= -1
    # n_jobs를 -1로 두었을 때 Remote Error가 발생하면 CPU의 과부하를 뜻하는 것이므로 해당 매개변수를 제거
)

# 6-2. 그리드 서치 - 분류 모델

In [32]:
grid_logi.fit(X_train, Y_train)

Fitting 4 folds for each of 16 candidates, totalling 64 fits


c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\joblib\externals\loky\process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\student\AppData\Local\Programs\Python\P

In [35]:
print(grid_logi.best_params_)
print(grid_logi.best_score_)

{'clf__C': 1.0, 'clf__max_iter': 800, 'vector__min_df': 3, 'vector__ngram_range': (1, 2)}
0.7832177099066779


In [36]:
# GridSearch 모델 중 점수가 가장 높은 모델을 변수에 저장
best_model = grid_logi.best_estimator_

In [37]:
# test data를 이용하여 정확도, f1 score를 확인
best_pred = best_model.predict(X_test)

In [38]:
best_acc = accuracy_score(best_pred, Y_test)
print(round(best_acc, 4))

best_f1 = f1_score(best_pred, Y_test)
print(round(best_f1, 4))

0.7898
0.7825


In [39]:
# ratings_test.txt 파일 로드
df_test = pd.read_csv("../data/ratings_test.txt", sep='\t')

In [40]:
df_test.dropna(inplace=True)

In [41]:
test_x = df_test['document'].values
test_y = df_test['label'].values

In [42]:
best_pred_test = best_model.predict(test_x)

In [43]:
print(round(
    accuracy_score(best_pred_test, test_y), 4
))
print(round(
    f1_score(best_pred_test, test_y), 4
))
# 파라미터값, 모델(SVC->Logi)을 바꿨음에도 성능에 큰 차이 없음
# 벡터를 바꿔볼 수 있음 (Tfid -> ?)

0.784
0.7776
